Pre processing and modeling
Our data is clean, now we must prepare it for modeling by spliting the data into training, validation, and testing sets. Then scale the necessary columns for modeling. The modeling stage will explore multiple models, comparing their ability to understand the data. We will also continue eda by exploring how the model made their classifications. The most appropriate model will then be tuned in hopes to improve performance, and then ultimately judged on its results from the test set.

In [ ]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scikitplot as skplt
import shap

from scipy.stats import randint, uniform

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, precision_recall_curve, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.calibration import CalibrationDisplay, calibration_curve

from xgboost import XGBClassifier

In [ ]:
# final/ current state of data
df.info()

In [ ]:
# breakdown the data into 3 sections: 70% train, 15% validation, 15% test

# create splits for features (X_) and target (y_)
X = df.drop('Churn', axis=1)
y = df['Churn']

# Initial split, creates 70% training sets, then a bulk 30% valid and test
X_train, X_BULK, y_train, y_BULK = train_test_split(X, y, test_size=0.3, random_state=100, stratify=y)

# break the bulk apart and create valid and test splits
X_valid, X_test, y_valid, y_test = train_test_split(X_BULK, y_BULK, test_size=0.5, random_state=100, stratify=y_BULK)

In [ ]:
# scale the continuous numerical columns: 'MonthlyCharges', 'MemberLengthDays'
num_col = ['MonthlyCharges', 'MemberLengthDays']

# load scaler
scaler = StandardScaler()

# fit on training, transform everything
X_train_scaled = scaler.fit_transform(X_train[num_col])
X_valid_scaled = scaler.transform(X_valid[num_col])
X_test_scaled = scaler.transform(X_test[num_col])

# make a dataframe out of the transformed columns
X_train_df = pd.DataFrame(X_train_scaled, columns=num_col, index=X_train.index)
X_valid_df = pd.DataFrame(X_valid_scaled, columns=num_col, index=X_valid.index)
X_test_df= pd.DataFrame(X_test_scaled, columns=num_col, index=X_test.index)

# combine transformed columns dataframe with the rest of the dataset
X_train_ids = pd.concat([X_train_df, X_train.drop(columns=num_col)], axis=1)
X_valid_ids = pd.concat([X_valid_df, X_valid.drop(columns=num_col)], axis=1)
X_test_ids = pd.concat([X_test_df, X_test.drop(columns=num_col)], axis=1)

# remove the unique identifier column
X_test_final = X_test_ids.drop(columns='CustomerID')
X_valid_final = X_valid_ids.drop(columns='CustomerID')
X_train_final = X_train_ids.drop(columns='CustomerID')

In [ ]:
# build function to train models storing results in a table/chart

def models(model, X_train, X_test, y_train, y_test):
    """
    loads the model, fits and predicts on the data provided
    
    args: 
    model is a list of tuples. each tuple contains the name of the model paired with the load command
    X_train and X_test pandas dataframe containing previous split
    y_train and y_test are series objects also created from the train test split
    
    returns: a print out of the performance metrics for the model"""
    results = []
    
    for name, mod in model:
        start_time = time.time()

        # train
        print(f"Training {name}")
        mod.fit(X_train, y_train)

        # predict
        y_pred = mod.predict(X_test)
        y_prob = mod.predict_proba(X_test)[:,1]

        # calculate metrics
        metrics = {
            'Model': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1-Score': f1_score(y_test, y_pred),
            'ROC AUC': roc_auc_score(y_test, y_prob),
            'Training Time (s)': time.time() - start_time
        }
        results.append(metrics)
    print("Model evaluation complete.")
    return pd.DataFrame(results).round(4).sort_values(by='ROC AUC', ascending=False)
    

In [ ]:
# list of models to test out
models_list = [('Logistic Regression', LogisticRegression(solver='liblinear', 
                                                          random_state=36)),
               ('Random Forest Classifier', RandomForestClassifier(n_estimators=100, 
                                                                   class_weight='balanced', 
                                                                   random_state=36)),
                ('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=36))]

In [ ]:
models(models_list, X_train_final, X_valid_final, y_train, y_valid)


Training Logistic Regression
Training Random Forest Classifier
Training XGBoost
Model evaluation complete.
Model	Accuracy	Precision	Recall	F1-Score	ROC AUC	Training Time (s)
2	XGBoost	0.8570	0.7611	0.6714	0.7135	0.8975	1.1761
0	Logistic Regression	0.7917	0.6250	0.5357	0.5769	0.8357	0.0189
1	Random Forest Classifier	0.7983	0.6558	0.5036	0.5697	0.8302	0.4749


thoughts
___
All three models performed well. Either the problem was simple to solve or our cleaning and processing of data was done well. Random Forest had the worst results, we will stop exploring it now. Logistic regression did well and for models it is more transparent as to why predictions are made. XGBoost did great already. an Auc-roc score of near 0.89 out of the box is great.

In [ ]:
# explore logistic regression prediction coeficients
# same model from before
log_reg = LogisticRegression(solver='liblinear', random_state=36)
log_reg.fit(X_train_final, y_train)

# store coefficients and feature names
coefficients = log_reg.coef_[0]
feature_names = X_train_final.columns

# turn into a dataframe for viewing
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients,
    'Absolute_Coefficient': np.abs(coefficients)})

# add odds_ratio
coef_df['Odds_Ratio'] = np.exp(coef_df['Coefficient'])

# display the dataframe
print('Logistic regression coefficients and odds ratios')
print('-'*80)
coef_df.sort_values(by='Absolute_Coefficient', ascending=False)

thoughts
___
Previously we noticed that those who have churned had higher average monthly bills. Based on the Logistic regression coefficients, monthly charges had the highest positive association to churning. Monthly charges was one of the standard scaled features, with an odds_ratio of 2.009 that means that for each unit increase (in this case the unit is standard deviation) the odds of churning increases by 100% or simply, it doubles.

Our other high positive association features were one hot encoded. So the odds ratio of 1.416 for month to month contracts simply means theres a 41.6% greater odds of churning compared to others plans.

Paperless Billing actually contributes to churn, this might tell you more about the type of customer who accepts this feature. Most of my personal bills offer a discount for paperless billing. Smaller monthly bills retain customers, so these two features appear to be counter-intuitive 

The two greatest predictors or member retension are two year contracts and member length in days. This makes sense, being locked into a two year contract prevents churn for some time, and during that time member length in days goes up. 

In [ ]:
# original data, unscaled essentially
df['MonthlyCharges'].describe()



thoughts
____
A monthly bill increase of \\$30.09 will double the odds of being churned. That is actually quite a large increase. The average bill is \\$64.74,  I do not know specifically what could cost \\$30 a month. 

We still havent tuned models, in order to more accurately identify churn candidates, however we have uncovered that bigger bills are leading to churn. It is important for InterConnect to investigate which services, add ons or bundling is leading to high bill averages. Potentially polling customers (data collection) on the preceived value of their contract. Are customers with higher bills being upsold into these contracts that they ultimately can't wait to get out of?  

We already know InterConnect has plans to be proactive in offering discounts or some sort of promotion to clients who are at risk of churn. That is this whole project. We also have identified that Internet Security and Tech support contribute to customer retension, offering a decrease in churn odds of 43% and 41% respectfully. They are high value features. We do not have information about the opperating costs associated with these features, consider offering these features as promotions

In [ ]:
# tuning XGBoost
# record run time
start_time = time.time()

# initialize the model
boost_model = XGBClassifier(use_label_encoder=False, 
                            eval_metric='logloss', 
                            random_state=36,
                           )
# create a parameter dictionary for random search cv, using ranges for each one to 
params = {
    'n_estimators': randint(100, 400),
    'learning_rate': uniform(0.1, 0.08),
    'max_depth': randint(4, 7),
    'subsample': uniform(0.8, 0.2),            
    'colsample_bytree': uniform(0.7, 0.3),     
    'gamma': uniform(0.1, 0.2),                 
    'lambda': uniform(0.8, 0.4),   
    'alpha': uniform(0, 0.2)     
}

# set up random search settings, grading by our desired metric (auc-roc)
random_search = RandomizedSearchCV(estimator=boost_model,
                                  param_distributions=params,
                                  n_iter=100,
                                  scoring='roc_auc',
                                  cv=5,
                                  verbose=1,
                                  random_state=36,
                                  n_jobs=1)


# fit to find the best params
random_search.fit(X_train_final, y_train)

# record stop run time
end_time = time.time()

# display best parameters abnd best results
print(f'Run time: {end_time-start_time}\n with the best params of:')
print(random_search.best_params_)
print()
print(f'Best result: {random_search.best_score_:.2f}')

# store the best model settings
best_xgb = random_search.best_estimator_

In [ ]:
# use the best model settings to make predictions on the validation set

# record run time
start_eval_time = time.time()

# store churn predictions (0,1)
y_pred_tuned_xgb_valid = best_xgb.predict(X_valid_final)

# store the churn probability (0.0 - 1.0), these are the models calculated probability of churn, default threshold is 0.5, above = churn, below no-churn
y_prob_tuned_xgb_valid = best_xgb.predict_proba(X_valid_final)[:, 1]

# results/ metrics
tuned_xgb_valid_metrics = {
    'Accuracy': accuracy_score(y_valid, y_pred_tuned_xgb_valid),
    'Precision': precision_score(y_valid, y_pred_tuned_xgb_valid),
    'Recall': recall_score(y_valid, y_pred_tuned_xgb_valid),
    'F1-Score': f1_score(y_valid, y_pred_tuned_xgb_valid),
    'ROC AUC': roc_auc_score(y_valid, y_prob_tuned_xgb_valid)
}

print("Tuned XGBoost Model Metrics (on Validation Set):")
for metric, value in tuned_xgb_valid_metrics.items():
    print(f"{metric}: {value:.4f}")

# record end run time
print(f"\nEvaluation completed in {(time.time() - start_eval_time):.4f} seconds.")


In [ ]:
# store the random search parametes tested, just in case
random_results_df = pd.DataFrame(random_search.cv_results_)

random_results_df.sort_values(by='rank_test_score', ascending=True)

thoughts
____
We now have a tuned model that is continueing to perform well on unseen data. The AUC-ROC score is 0.9, the recall is only 0.68 though. Recall is important for this problem because the churn category was the positive category. Recall is correctly predicted churns ( True positives ) divided by the total of correctly predicted churns ( true positves again ) + churners that the model missed ( false negatives). For this business problem the downside of miss labeling a non-churn candidate as churn-able, simply means we offered an extra discount/ promotion (slight decrease in revenue) while missing someone who would churn is a greater decrease in revenue.

We are able to adjust the classification threshold to purposely predict more churn. This should increase False positives (people who receive promotions etc.) and decrease False Negatives (People who churn without receiving a promotion). InterConnect would need to provide more information on how they calculate customer lifetime value or what would be remaining based on their current memberlength. Expected success rate of promotion etc. Combined they could plot the predicted cost to the company at each possible threshold.

In [ ]:
# test our best model's performance on the unseen data

# record run time
start_eval_time = time.time()

# store predictions and prediction probabilities
y_pred_tuned_xgb_test = best_xgb.predict(X_test_final)
y_prob_tuned_xgb_test = best_xgb.predict_proba(X_test_final)[:, 1] 

# results/ metrics dictionary
tuned_xgb_test_metrics = {
    'Accuracy': accuracy_score(y_test, y_pred_tuned_xgb_test),
    'Precision': precision_score(y_test, y_pred_tuned_xgb_test),
    'Recall': recall_score(y_test, y_pred_tuned_xgb_test),
    'F1-Score': f1_score(y_test, y_pred_tuned_xgb_test),
    'ROC AUC': roc_auc_score(y_test, y_prob_tuned_xgb_test)
}

# display results
print("Tuned XGBoost Model Metrics (on TEST Set):")
for metric, value in tuned_xgb_test_metrics.items():
    print(f"{metric}: {value:.4f}")
    
# end run time
print(f"\nEvaluation completed in {(time.time() - start_eval_time):.4f} seconds.")


thoughts
____
The model performed very similiar. This is great, results on unseen data are expected to be slightly worse. Recall did decrease slightly, however we will explore that shortly.

In [ ]:
# display prediction results with through a confusion matrix (TP, TN, FP, FN)

# plot the confusion matrix with labels for easier reading
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix(y_test, y_pred_tuned_xgb_test),
            annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted No Churn', 'Predicted Churn'],
            yticklabels=['Actual No Churn', 'Actual Churn'])
plt.title(f'Confusion Matrix on Test Set')
plt.ylabel('Actual Label')
plt.xlabel('Predicted Label')
plt.show()


thoughts
___
We knew the performance metrics we decent which means the True Positive and True Negatives were going to be the largest groups (correct predictions). However our specific business problem isn't acknowledged by the model. 

- This data contained 281 accounts that churned.
- correctly identified 65% of them, so 35% wouldn't receive promotions, not great.
- the model identified 238 accounts suitable for churn (promotions), and 22.7% of promotions went to accounts that had not churned AT THIS TIME. just because they hadn't churned doesn't mean they weren't about it.

In [ ]:
# plot precision recall curve, 
plt.figure(figsize=(8, 6))
precision, recall, thresholds_pr = precision_recall_curve(y_test, y_prob_tuned_xgb_test)
plt.plot(recall, precision, label='Precision-Recall curve')
plt.plot([0.6548, 0.6548], [0.0, 1], color='red', lw=2, linestyle='--', label='Recall at default threshold')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall curve on test set')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()


thoughts
____
From the percision recall curve, we can see that by increasing the recall metric we will see a drop in percision. This is normal. How much percision are we willing to lose?

Percision (y axis) high percision means our targeting effort is efficient. No-or-fewer unnecessary promotions.

Recall (x axis) high recall means everyone in danger of churning gets a promotion, and the error is extra promotions. I believe this is the better option, however if InterConnect doesn't like to appearance of being the 'coupon' company, or the discount-guys, then high percision would be important to them.

In [ ]:
# ROC Curve
plt.figure(figsize=(8, 6))
fpr, tpr, thresholds_roc = roc_curve(y_test, y_prob_tuned_xgb_test)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, color='orange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.plot([0, 1], [0.6548, 0.6548], color='red', lw=2, linestyle='--', label="Model's Recall")
plt.xlabel('False positive rate')
plt.ylabel('True positive rate (Recall)')
plt.title('Receiver operating characteristic (ROC) curve on test set')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


thoughts
____
From the ROC curve we can see that our model is performing a lot better than a random model. We already knew this. We can also see that adjusting the prediction threshold could increase recall. The ROC apex is above the current threshold position. Meaning we can increase recall faster than the increase of false positives. Capturing churners faster than offering extra discounts.

In [ ]:
# create a dataframe with all the test information, results, probabilities
combined_test = X_test_final.copy()
combined_test['Actual'] = y_test
combined_test['Prediction'] = y_pred_tuned_xgb_test
combined_test['Probability'] = y_prob_tuned_xgb_test

# create subsets for FN FP TN TP
tp = combined_test[(combined_test['Actual'] == 1) & (combined_test['Prediction'] == 1)]
tn = combined_test[(combined_test['Actual'] == 0) & (combined_test['Prediction'] == 0)]
fp = combined_test[(combined_test['Actual'] == 0) & (combined_test['Prediction'] == 1)]
fn = combined_test[(combined_test['Actual'] == 1) & (combined_test['Prediction'] == 0)]

In [ ]:
# probability histograms for predictions 

plt.figure(figsize=(15, 30))
bins = np.linspace(0, 1, 50)

# plot 1: True Positives and False Negatives (Actual Churners)
plt.subplot(4, 2, 1)
sns.histplot(tp['Probability'], bins=bins, color='green', label='True Positives', kde=True, stat='density', alpha=0.5)
sns.histplot(fn['Probability'], bins=bins, color='red', label='False Negatives', kde=True, stat='density', alpha=0.5)
plt.axvline(x=0.5, color='black', linestyle='--', label=f'Threshold ({0.5})')
plt.title('Actual Churners: TP vs FN Probability Distribution')
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--')

# plot 2: True Negatives and False Positives (Actual Non-Churners)
plt.subplot(4, 2, 2)
sns.histplot(tn['Probability'], bins=bins, color='blue', label='True Negatives', kde=True, stat='density', alpha=0.5)
sns.histplot(fp['Probability'], bins=bins, color='orange', label='False Positives', kde=True, stat='density', alpha=0.5)
plt.axvline(x=0.5, color='black', linestyle='--', label=f'Threshold ({0.5})')
plt.title('Actual Non-Churners: TN vs FP Probability Distribution')
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--')

# plot 3: All Correct Predictions (TP + TN)
plt.subplot(4, 2, 3)
sns.histplot(tp['Probability'], bins=bins, color='green', label='True Positives', kde=True, stat='density', alpha=0.5)
sns.histplot(tn['Probability'], bins=bins, color='blue', label='True Negatives', kde=True, stat='density', alpha=0.5)
plt.axvline(x=0.5, color='black', linestyle='--', label=f'Threshold ({0.5})')
plt.title('Correct Predictions: TP & TN Probability Distribution')
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--')

# plot 4: All Incorrect Predictions (FP + FN)
plt.subplot(4, 2, 4)
sns.histplot(fp['Probability'], bins=bins, color='orange', label='False Positives', kde=True, stat='density', alpha=0.5)
sns.histplot(fn['Probability'], bins=bins, color='red', label='False Negatives', kde=True, stat='density', alpha=0.5)
plt.axvline(x=0.5, color='black', linestyle='--', label=f'Threshold ({0.5})')
plt.title('Incorrect Predictions: FP & FN Probability Distribution')
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--')

# plot 5: All Negative Predictions (TN + FN)
plt.subplot(4, 2, 5)
sns.histplot(tn['Probability'], bins=bins, color='blue', label='True Negatives', kde=True, stat='density', alpha=0.5)
sns.histplot(fn['Probability'], bins=bins, color='red', label='False Negatives', kde=True, stat='density', alpha=0.5)
plt.axvline(x=0.5, color='black', linestyle='--', label=f'Threshold ({0.5})')
plt.title('Negative Predictions: TN & FN Probability Distribution')
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--')


# plot 6: All Positive Predictions (TP + FP)
plt.subplot(4, 2, 6)
sns.histplot(tp['Probability'], bins=bins, color='green', label='True Positives', kde=True, stat='density', alpha=0.5)
sns.histplot(fp['Probability'], bins=bins, color='orange', label='False Positives', kde=True, stat='density', alpha=0.5)
plt.axvline(x=0.5, color='black', linestyle='--', label=f'Threshold ({0.5})')
plt.title('Positive Predictions: TP & FP Probability Distribution')
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--')

# plot 7: All Predictions
plt.subplot(4, 2, 7)
sns.histplot(tp['Probability'], bins=bins, color='green', label='True Positives', kde=True, stat='density', alpha=0.5)
sns.histplot(fp['Probability'], bins=bins, color='orange', label='False Positives', kde=True, stat='density', alpha=0.5)
sns.histplot(tn['Probability'], bins=bins, color='blue', label='True Negatives', kde=True, stat='density', alpha=0.5)
sns.histplot(fn['Probability'], bins=bins, color='red', label='False Negatives', kde=True, stat='density', alpha=0.5)
plt.axvline(x=0.5, color='black', linestyle='--', label=f'Threshold ({0.5})')
plt.title('All Prediction Probabilities')
plt.xlabel('Predicted Probability of Churn')
plt.ylabel('Density')
plt.legend()
plt.grid(axis='y', linestyle='--')



plt.tight_layout()
plt.show()

thoughts
___
- Correct Predictions: probabilities were most decisive, the spread grows away from the threshold
- Incorrect Predictions: noisy histogram. False positives (customers predicted to churn, that have not yet) has higher density around the threshold.
- False Negatives: those who churned that the model missed, these are the most important for increasing profit. These are customers who would not receive a promotion.
- Even without knowing how InterConnect calculates lifetime value for customers, we can see (plot 5: all negative predictions) if our threshold became 0.3 for example we would capture a lot more of the False Negatives than providing extra discounts to non-churners.
- Going more extreme with threshold adjustments, the kde lines cross at about 0.125.
- Because essentially all the noise in the incorrect predictions, theres potential to do some more feature engineering. 

Conclusion:
____
We set out to develop a predictive model for customer churn to help InterConnect retain at risk customers. I've successfully constructed and extensively tuned an XGBoost model to predict churn for InterConnect. 

The models performance on test set (probability threshold 0.5): 
Accuracy: 0.8571, the model correctly identified 85.7% of client's status
Precision: 0.7731, 77.3% of churn predictions were correct
Recall: 0.6548, 65.5% of actual churners were identified 
F1-Score: 0.7091, a balance of precision and recall
ROC AUC: 0.8988, models ability to distinguish between the two classes, 1 is perfect

The model was able to confidently predict TP TN. The histogram of probabilities from these classes were highly skewed. There was some noise when it came to the incorrect predictions. Noise could be removed with further feature engineering. Even with the noise, a different threshold can be chosen by InterConnect based on how they calculate Customer Lifetime Value. We also do not know the specifics of the proposed promotion. Different promotions would lead to a different threshold. I suggest set the threshold to a number that provides the smallest decrease in revenue. Adjusting the threshold down, would increase recall at the expense of precision. The model would begin to err on the side of predicting churn. This would increase the number of promotions sent out.

The model is technically performing well, but it is only as useful as it's ability to retain customers. We don't have information on who used tech support, if those calls/ session were positive or not. Has a customer missed a bill? I am sure InterConnect is aware humans are human, so they are only so predictable. Does this model fit the scale of marketing campaign InterConnect had in mind? i.e. were they thinking of offering hundreds of discounts? thousands? We can continue to work together on this. 